In [1]:
from tensornetworks_pytorch_combined.TNModels import PositiveMPS

In [5]:
mps = PositiveMPS(D=2, d=8, verbose=True) 

In [6]:
list(mps.parameters())[1]

Parameter containing:
tensor([0.2867, 0.8956], requires_grad=True)

In [7]:
mps.left_boundary.square()

tensor([0.0822, 0.8020], grad_fn=<PowBackward0>)

In [8]:
import pickle
with open('datasets/lymphography', 'rb') as f:
        a=pickle.load(f)
X=a[0]
X=X.astype(int)
X.max()

7

In [9]:
X[0]

array([2, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 7, 1, 1, 1, 0])

In [10]:
mps.fit(X, 8)
mps.core

Parameter containing:
tensor([[[0.2862, 0.9039],
         [0.5952, 0.3897]],

        [[0.8965, 0.4357],
         [0.2663, 0.0897]],

        [[0.8735, 0.4744],
         [0.8929, 0.0818]],

        [[0.3030, 0.8852],
         [0.2803, 0.5673]],

        [[0.1825, 0.5100],
         [0.0782, 0.3894]],

        [[0.7709, 0.5564],
         [0.8503, 0.9044]],

        [[0.6224, 0.0692],
         [0.5959, 0.6292]],

        [[0.2410, 0.5793],
         [0.3687, 0.5706]]], requires_grad=True)

In [11]:
print("n_features", mps.n_features)
print("norm", mps.norm)
print("prob0",mps.probability0)

n_features 19
norm tensor(6.8725e+16, grad_fn=<ViewBackward>)
prob0 tensor(0.0001, grad_fn=<ViewBackward>)


In [12]:
mps.core.abs()

tensor([[[0.2862, 0.9039],
         [0.5952, 0.3897]],

        [[0.8965, 0.4357],
         [0.2663, 0.0897]],

        [[0.8735, 0.4744],
         [0.8929, 0.0818]],

        [[0.3030, 0.8852],
         [0.2803, 0.5673]],

        [[0.1825, 0.5100],
         [0.0782, 0.3894]],

        [[0.7709, 0.5564],
         [0.8503, 0.9044]],

        [[0.6224, 0.0692],
         [0.5959, 0.6292]],

        [[0.2410, 0.5793],
         [0.3687, 0.5706]]], grad_fn=<AbsBackward>)